Consider $s$ parallel servers.
- If a customer arrives and finds an idle server, they begin service at the leftmost idle server.
- If a customer finds no server idle, they enter a single FIFO queue.

In [2]:
import numpy as np

def expon(mean): 
    return np.random.exponential(scale=mean)

In [49]:
MEAN_INTERARRIVAL = 1.0
MEAN_SERVICE = 4.0

NUM_SERVERS = 5
NUM_EVENTS = 6

# sim clock
SIM_TIME = 0.0

# queue
TIME_ARRIVAL = []

# event list
TIME_NEXT_EVENT = [
    SIM_TIME + expon(MEAN_INTERARRIVAL), # arrival
    np.inf, # depart 1
    np.inf, # depart 2
    np.inf, # depart 3
    np.inf, # depart 4
    np.inf, # depart 5
    ] # arrive, depart1, depart2, terminate
TIME_LAST_EVENT = 0.0

# state vars
NUM_IN_Q = 0
SERVER_STATUS = [0] * NUM_SERVERS # 0 = idle; 1 = busy

# stats
NUM_CUSTS_DELAYED = 0
TOTAL_OF_DELAYS = 0.0
AREA_NUM_IN_Q = 0.0
AREA_SERVER_STATUS = [0.0] * NUM_SERVERS


def arrive():
    """A customer arrives to the facility.
    
    If there are available servers, customer proceeds immediately to the first available server.
    If there are no available servers, customer enters the queue.

    The next arrival is scheduled.
    """
    global SERVER_STATUS, SIM_TIME, TIME_NEXT_EVENT
    
    # find an available server
    available_server = -1
    for idx, val in enumerate(SERVER_STATUS):
        if val == 0:
            available_server = idx
            break
    
    if available_server != -1: # there is an available server
        # go directly to service at this server
        delay = 0.0
        SERVER_STATUS[available_server] = 1 # make busy
        global TOTAL_OF_DELAYS, NUM_CUSTS_DELAYED
        TOTAL_OF_DELAYS += delay
        NUM_CUSTS_DELAYED += 1 # increment customer delay count

        # schedule departure event
        global MEAN_SERVICE
        TIME_NEXT_EVENT[1 + available_server] = SIM_TIME + expon(MEAN_SERVICE)

    else:
        # all servers are busy, so enter queue
        global NUM_IN_Q
        NUM_IN_Q += 1

        # store arrival time to calculate delay later
        global TIME_ARRIVAL
        TIME_ARRIVAL += [SIM_TIME]
    
    # schedule next arrival
    global MEAN_INTERARRIVAL
    TIME_NEXT_EVENT[0] = SIM_TIME + expon(MEAN_INTERARRIVAL)


def depart(s: int):
    """A customer departs from Server s (0-4)."""
    global NUM_IN_Q
    if NUM_IN_Q == 0:
        # queue is empty, so server becomes idle
        global SERVER_STATUS, TIME_NEXT_EVENT
        SERVER_STATUS[s] = 0
        TIME_NEXT_EVENT[s+1] = np.inf
    else:
        # there are customers in the queue
        NUM_IN_Q -= 1

        # compute delay of customer starting service
        global SIM_TIME, TIME_ARRIVAL, TOTAL_OF_DELAYS
        delay = SIM_TIME - TIME_ARRIVAL[0]
        TOTAL_OF_DELAYS += delay

        # increment number of delayed customers
        global NUM_CUSTS_DELAYED
        NUM_CUSTS_DELAYED += 1

        # schedule departure from server s
        global MEAN_SERVICE
        TIME_NEXT_EVENT[s+1] = SIM_TIME + expon(MEAN_SERVICE)

        # shift queue
        TIME_ARRIVAL = TIME_ARRIVAL[1:]


def timing():
    global NEXT_EVENT_TYPE, NUM_EVENTS, TIME_NEXT_EVENT, SIM_TIME
    min_time_next_event = np.inf

    for i in range(NUM_EVENTS):
        if TIME_NEXT_EVENT[i] < min_time_next_event:
            min_time_next_event = TIME_NEXT_EVENT[i]
            NEXT_EVENT_TYPE = i

    SIM_TIME = TIME_NEXT_EVENT[NEXT_EVENT_TYPE]


def update_time_avg_stats():
    """Prior to processing each event, this function updates the areas tracked
    for the calculation of continuous-time statistics."""
    global SIM_TIME, TIME_LAST_EVENT
    time_since_last_event = SIM_TIME - TIME_LAST_EVENT
    TIME_LAST_EVENT = SIM_TIME # current time

    # update area under num-in-queue function
    global AREA_NUM_IN_Q, NUM_IN_Q
    AREA_NUM_IN_Q += NUM_IN_Q * time_since_last_event

    # update area under server-busy indicator function
    global AREA_SERVER_STATUS, SERVER_STATUS
    for idx, ss in enumerate(SERVER_STATUS):
        AREA_SERVER_STATUS[idx] += ss * time_since_last_event

def report():
    """Log statistics"""
    global TOTAL_OF_DELAYS, NUM_CUSTS_DELAYED, AREA_NUM_IN_Q, SIM_TIME, AREA_SERVER_STATUS
    print(f"\n\nAverage delay in queue {TOTAL_OF_DELAYS / NUM_CUSTS_DELAYED:11.3f} minutes\n\n")
    print(f"Average number in queue {AREA_NUM_IN_Q / SIM_TIME:10.3f}\n\n")
    for ss_idx, ss in enumerate(AREA_SERVER_STATUS):
        print(f"Server {ss_idx + 1} utilization {ss / SIM_TIME:15.3f}\n\n")
    print(f"Time simulation ended {SIM_TIME:12.3f} minutes")

while True:
    if NUM_CUSTS_DELAYED == 1000:
        report()
        break
    timing()
    # print(TIME_NEXT_EVENT)

    update_time_avg_stats()
    match NEXT_EVENT_TYPE:
        case 0:
            arrive()
        case _:
            depart(s=NEXT_EVENT_TYPE-1)
        



Average delay in queue       2.038 minutes


Average number in queue      2.111


Server 1 utilization           0.864


Server 2 utilization           0.824


Server 3 utilization           0.773


Server 4 utilization           0.689


Server 5 utilization           0.660


Time simulation ended      990.698 minutes
